In [2]:
import json

j = '''
{
    "a": 100,
    "b": [1, 2, 3],
    "c": "Python",
    "d": {
        "e": 4,
        "f": 5.5
    }
}
'''

class CustomDecoder(json.JSONDecoder):
    def decode(self, arg):
        print(f"Decoding:{type(arg)}, {arg}")
        return "a simple string object"
    
json.loads(j, cls=CustomDecoder)

Decoding:<class 'str'>, 
{
    "a": 100,
    "b": [1, 2, 3],
    "c": "Python",
    "d": {
        "e": 4,
        "f": 5.5
    }
}



'a simple string object'

In [17]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f"Point(x={self.x}, y={self.y})"
    
j_points = '''
{   
    "points": [
        [10, 20],
        [-1, -2],
        [0.5, 0.5]
    ]
}
'''

j_other = '''
{
    "a": 1,
    "b": 2
}
'''

class CustomDecoder(json.JSONDecoder):
    def decode(self, arg):
        if "points" in arg:
            obj = json.loads(arg)
            return "parsing object for points"
        else:
            return super().decode(arg)

json.loads(j_points, cls=CustomDecoder)

'parsing object for points'

In [18]:
json.loads(j_other, cls=CustomDecoder)

{'a': 1, 'b': 2}

In [19]:
class CustomDecoder(json.JSONDecoder):
    def decode(self, arg):
        obj = json.loads(arg)
        if "points" in obj:
            obj['points'] = [Point(x, y) for x, y in obj['points']]
        return obj
    
json.loads(j_points, cls=CustomDecoder)

{'points': [Point(x=10, y=20), Point(x=-1, y=-2), Point(x=0.5, y=0.5)]}

In [20]:
json.loads(j_other, cls=CustomDecoder)

{'a': 1, 'b': 2}

In [21]:
json.loads(j, cls = CustomDecoder)

{'a': 100, 'b': [1, 2, 3], 'c': 'Python', 'd': {'e': 4, 'f': 5.5}}

In [24]:
import re

pattern = r'"_type"\s*:\s*"point"'

regexp = re.compile(pattern)

print(regexp.search(j_points))

None


In [26]:
print(regexp.search('"_type"        : "point"'))

<re.Match object; span=(0, 24), match='"_type"        : "point"'>


In [27]:
re.search(pattern, '"_type"        : "point"')

<re.Match object; span=(0, 24), match='"_type"        : "point"'>

In [32]:
class CustomDecoder(json.JSONDecoder):
    def decode(self, arg):
        obj = json.loads(arg)
        pattern = r'"_type"\s*:\s*"point"'
        if re.search(pattern, arg):
            obj = self.make_pts(obj)
        return obj
    
    def make_pts(self, obj):
        if isinstance(obj, dict):
            if obj.get("_type", None) == "point":
                obj = Point(obj['x'], obj['y'])
            else:
                for key, value in obj.items():
                    obj[key] = self.make_pts(value)

        elif isinstance(obj, list):
            for index, item in enumerate(obj):
                obj[index] = self.make_pts(item)
        return obj
    
j = '''
{
    "a": 100,
    "b": 0.5,
    "rectangle": {
        "corners": {
            "b_left": {"_type": "point", "x": -1, "y": -1},
            "b_right": {"_type": "point", "x": 1, "y": -1},
            "t_left": {"_type": "point", "x": -1, "y": 1},
            "t_right": {"_type": "point", "x": 1, "y": 1}
        },
        "rotate": {"_type": "point", "x": 0, "y": 0},
        "interior_pts": [
            {"_type": "point", "x": 0, "y": 0},
            {"_type": "point", "x": 0.5, "y": 0.5}
        ]
    }
}
'''


json.loads(j)

{'a': 100,
 'b': 0.5,
 'rectangle': {'corners': {'b_left': {'_type': 'point', 'x': -1, 'y': -1},
   'b_right': {'_type': 'point', 'x': 1, 'y': -1},
   't_left': {'_type': 'point', 'x': -1, 'y': 1},
   't_right': {'_type': 'point', 'x': 1, 'y': 1}},
  'rotate': {'_type': 'point', 'x': 0, 'y': 0},
  'interior_pts': [{'_type': 'point', 'x': 0, 'y': 0},
   {'_type': 'point', 'x': 0.5, 'y': 0.5}]}}

In [34]:
from pprint import pprint
pprint(json.loads(j, cls=CustomDecoder))

{'a': 100,
 'b': 0.5,
 'rectangle': {'corners': {'b_left': Point(x=-1, y=-1),
                           'b_right': Point(x=1, y=-1),
                           't_left': Point(x=-1, y=1),
                           't_right': Point(x=1, y=1)},
               'interior_pts': [Point(x=0, y=0), Point(x=0.5, y=0.5)],
               'rotate': Point(x=0, y=0)}}


In [37]:
from decimal import Decimal

CustomDecoder = json.JSONDecoder(parse_float=Decimal)

pprint(json.loads(j, cls=CustomDecoder))

TypeError: 'JSONDecoder' object is not callable

In [36]:
d

{'a': 100,
 'b': Decimal('0.5'),
 'rectangle': {'corners': {'b_left': {'_type': 'point', 'x': -1, 'y': -1},
   'b_right': {'_type': 'point', 'x': 1, 'y': -1},
   't_left': {'_type': 'point', 'x': -1, 'y': 1},
   't_right': {'_type': 'point', 'x': 1, 'y': 1}},
  'rotate': {'_type': 'point', 'x': 0, 'y': 0},
  'interior_pts': [{'_type': 'point', 'x': 0, 'y': 0},
   {'_type': 'point', 'x': Decimal('0.5'), 'y': Decimal('0.5')}]}}

In [39]:
class CustomDecoder(json.JSONDecoder):
    base_decoder = json.JSONDecoder(parse_float=Decimal)

    def decode(self, arg):
        obj = self.base_decoder.decode(arg)
        pattern = r'"_type"\s*:\s*"point"'
        if re.search(pattern, arg):
            obj = self.make_pts(obj)
        return obj
    
    def make_pts(self, obj):
        if isinstance(obj, dict):
            if obj.get("_type", None) == "point":
                obj = Point(obj['x'], obj['y'])
            else:
                for key, value in obj.items():
                    obj[key] = self.make_pts(value)

        elif isinstance(obj, list):
            for index, item in enumerate(obj):
                obj[index] = self.make_pts(item)
        return obj
    

pprint(json.loads(j, cls=CustomDecoder))

{'a': 100,
 'b': Decimal('0.5'),
 'rectangle': {'corners': {'b_left': Point(x=-1, y=-1),
                           'b_right': Point(x=1, y=-1),
                           't_left': Point(x=-1, y=1),
                           't_right': Point(x=1, y=1)},
               'interior_pts': [Point(x=0, y=0), Point(x=0.5, y=0.5)],
               'rotate': Point(x=0, y=0)}}


In [43]:
result = json.loads(j, cls = CustomDecoder)
pt = result['rectangle']['interior_pts'][1]
type(pt.x)

decimal.Decimal

In [44]:
class CustomDecoder(json.JSONDecoder):
    def __init__(self, *args, **kwargs):
        super().__init__(parse_float=Decimal)
    base_decoder = json.JSONDecoder(parse_float=Decimal)

    def decode(self, arg):
        obj = super().decode(arg)
        pattern = r'"_type"\s*:\s*"point"'
        if re.search(pattern, arg):
            obj = self.make_pts(obj)
        return obj
    
    def make_pts(self, obj):
        if isinstance(obj, dict):
            if obj.get("_type", None) == "point":
                obj = Point(obj['x'], obj['y'])
            else:
                for key, value in obj.items():
                    obj[key] = self.make_pts(value)

        elif isinstance(obj, list):
            for index, item in enumerate(obj):
                obj[index] = self.make_pts(item)
        return obj
    

pprint(json.loads(j, cls=CustomDecoder))

{'a': 100,
 'b': Decimal('0.5'),
 'rectangle': {'corners': {'b_left': Point(x=-1, y=-1),
                           'b_right': Point(x=1, y=-1),
                           't_left': Point(x=-1, y=1),
                           't_right': Point(x=1, y=1)},
               'interior_pts': [Point(x=0, y=0), Point(x=0.5, y=0.5)],
               'rotate': Point(x=0, y=0)}}
